# Cartographie : le mouvement universitaire médiéval en Europe

Cette cartographie fait suite à la collection de données de Eric Guichard et Camille Ducrot.


### Regarder les données

Jetons rapidement un oeil sur les données.


In [1]:
# parse clean data using panda

import pandas as pd
import numpy as np

df = pd.read_csv(open("../data/universites.csv", "r"))

df = df.fillna(value="0")
df.head()

,Université,Date de fondation,1200,1225,1250,1270,1275,1300,1325,1350,...,1760,1764,1766,1770,1775,1780,1785,1790,1800,1810
0,Aberdeen,1494,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Abo,1640,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Aix-en-Provence,1409,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Alcalà de Henares,1499,0,0,0,0,0,0,0,0,...,M : 847,0,0,M : 783,0,M : 519,0,M : 492,M : 542,0
4,Almagro,1550,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
df_info = pd.read_csv(open("../data/universites-info.csv", "r"))
df_info = df_info.fillna(value="0")
df_info.head()

,Université,Nom francisé,Pays,Date de fondation,Date de confirmation,Date de fermeture,Réouverture,Fermeture,Réouverture.1,Fermeture.1,Confession,Origine,Créateur,Fac Théologie,Fac Médecine,Fac Droit Canon,Fac droit Civil,Fac Art/philo,Unnamed: 18
0,Aberdeen,Aberdeen,Ecosse,1494,Bulle papale : 1494,0,0,0,0,0,Catholique ; Prebysterien : 1560,Création ecclésiastique,"William Elphinstone, Evèque d'Aberdeen : 1494 ...",Ouv,Ouv,Ouv,Ouv,Ouv,0
1,Abo,"Turku, Finlande",Royaume de Suède,1640,0,0,0,0,0,0,Luthérien,Création princière,Pierre de Brahé,Ouv,Ouv,Ouv,Ouv,Ouv,0
2,Aix-en-Provence,0,Provence : 1409 ; Royaume de France : 1486,1409,Bulle papale : 1409,1793,0,0,0,0,Catholique,Création princière,Louis II d'Anjou,Ouv,Ouv : 1562,Ouv,Ouv,Ouv,0
3,Alcalà de Henares,0,Royaume des deux Espagnes : 1499 ; Royaume d'E...,1499,Bulle papale : 1499 ; Début : 1508 ; Bulle pap...,0,0,0,0,0,Catholique,Création ecclésiastique,Cardinal Jimenez de Cisneros,Ouv,Ouv : 1514,Ouv,Ouv : 1772,Ouv,0
4,Almagro,0,Royaume des deux Espagnes : 1550 ; Royaume d'E...,1550,Bulle papale : 1552,1807,0,0,0,0,Catholique,Création ecclésiastique,Couvent dominicain,Ouv,Abs,Abs,Abs,Ouv,0


## Géocodage 

Rertouvons latitutde et longitude depuis le nom de ville 

In [3]:
import json
import os
import csv
import geocoder


with open("../data/universites.csv", "r") as f:
    data = csv.reader(f)
    data.next()  # skip the headers
    
    villes = []
    for row in data:
        ville = row[0].decode('utf-8')
        villes.append(ville)

print "%s villes"%len(villes)

# platforms = ["google", "osm", "mapquest", "bing"]
platform = "google" 

geocodes = {}
geocode_file = "../data/geocodes-"+platform+".json"

# fetch data from the web
if not os.path.exists(geocode_file):
    print "fetching data online"

    for i,ville in enumerate(villes): 
        g = geocoder.bing(ville)
        geocodes[ville] = g.json
        print i, ville

        # save to file
        with open(geocode_file, "wb") as out:
            json.dump(geocodes, out, indent=2)

    print "%s/%s records geolocation with %s saved correcly"%(len(geocodes.keys()), i, platform) 

192 villes


### Détecter (et corriger) les erreurs de géocodage

* Tout d'abord, regardons les réponses des différents serveurs de géocodage interrogés.
* Ensuite, regardons la carte est constatons que plusieurs des points sont situés hors des frontières de l'Europe.


In [34]:
platform = "google"

print "="*10
print platform
print 


# read records
with open("../data/geocodes-" + platform + ".json", "r") as f:
    geocodes = json.load(f)
    print "%s geocoded records"%len(geocodes.keys())

    # check error
    errors = []

    for city in geocodes:
        g = geocodes[city]
        if "location" not in g.keys():
            pass # ignore empty value
        elif g["ok"] == False: 
            errors.append(g) # error during geocoding
        elif "country" in g.keys() and g["country"] in ["US", "CL", "AU", "JP"] : # using https://fr.wikipedia.org/wiki/ISO_3166-1
            errors.append(g)
        
print "-"*10
print "%s errors"%len(errors)
for error in errors: 
    print "* " + error["location"]
print "-"*10


google

188 geocoded records
----------
16 errors
* R : rotuli
* Dilligen
* Onasbrück
* Dublin
* Braunsberg
* Besançon
* Orange
* Sedan
* Nimeguen
* Cambridge
* Escorial
* Monbéliard
* Saint-Andrews
* Palma de Mallorca
* Würzbourg
* Cahors
----------


### 

In [35]:
### Manually add missing values

clean_geocodes = {}

to_be_removed = [ "R : rotuli", "J : collège jésuites", "M : matricules", "Est : Estimation"]

with open("../data/geocodes-errorfix.json", "r") as f:
    
    errorfix = json.load(f)
    print "%s error fix + %s to be removed"%(len(errorfix.keys()), len(to_be_removed))
    print errorfix.keys()
    
    for city in geocodes:
        g = geocodes[city]
        if "location" in g.keys() and city.encode('utf-8') not in to_be_removed and "rotuli" not in city:
            print g
            if city in errorfix.keys():
                clean_geocodes[city] = errorfix[city]
            else :
                clean_geocodes[city] = [g["lat"], g["lng"]]

print len(geocodes), len(clean_geocodes)

with open("../data/geocodes-final.json", "wb") as f:
    json.dump(clean_geocodes, f, indent=2)
    

16 error fix + 4 to be removed
[u'Cambridge', u'Palma de Mallorca', u'Dilligen', u'Besan\xe7on', u'Dublin', u'Onasbr\xfcck', u'Braunsberg', u'Sedan', u'Saint-Andrews', u'Nimeguen', u'Cahors', u'Escorial', u'Orange', u'Monb\xe9liard', u'Laguna', u'W\xfcrzbourg']
{u'status': u'OK', u'city': u'Kaliningrad', u'confidence': 1, u'ok': True, u'encoding': u'utf-8', u'country': u'RU', u'address': u'Kaliningrad, Kaliningrad Oblast, Russia', u'state': u'Kaliningrad Oblast', u'country_long': u'Russia', u'county': u'g. Kaliningrad', u'state_long': u'Kaliningrad Oblast', u'city_long': u'Kaliningrad', u'location': u'K\xf6nigsberg', u'provider': u'google', u'lat': 54.7104264, u'lng': 20.4522144, u'accuracy': u'APPROXIMATE', u'quality': u'locality', u'bbox': {u'northeast': [54.7786129, 20.633394], u'southwest': [54.644606, 20.2956661]}}
{u'status': u'OK', u'city': u'Pamplona', u'confidence': 5, u'ok': True, u'encoding': u'utf-8', u'country': u'ES', u'address': u'Pamplona, Navarre, Spain', u'state': u'N

In [31]:
%matplotlib inline
import matplotlib.pyplot as plt


# turn into time series
# can not be converted to time series - see http://stackoverflow.com/questions/32888124/pandas-out-of-bounds-nanosecond-timestamp-after-offset-rollforward-plus-adding-a


# remove M: from values
# for key in df :
#     if key not in ["Université", "Date de fondation"]: 
# #         print df[key]
#         df[key] = df[key].astype(np.str).str.replace("\s", "").str.replace("M", "").str.replace(":", "").astype(np.float)


In [91]:
universites = []

# parse clean data
with open("../data/universites.csv", "r") as f:
    data = csv.reader(f)
    
    for i, row in enumerate(data):
        if i == 0:
            years = [r for j, r in enumerate(row) if j > 1]
        else :
            univ = {
                "name" : row[0],
                "created" : row[1]
            }
            
            for j,year in enumerate(years):
                univ[year] = row[j]
            
            universites.append(univ)